# Thought experiments with SHAP

## Shapley value defintion
Given a set of n elements, $U := \{1, \cdots, n\}$ and a function

$g: 2^U \rightarrow \mathbb{R}$ with $g(U) \neq 0$, $g(\varnothing)=0$

define $C(j|T) := g(T \cup \{j\}) - g(T)$ with $T \subseteq U$.

Feture i's contribution:

$\phi_i := \sum_{T \subseteq U} \frac{|T|! (n-1-|T|)!}{n!} C(i|T)$

where
$g(T) := f_T(x_T) - f(x')$ with $f_\varnothing(x_\varnothing) = f(x')$ for some baseline $x'$. 

$f_T(x_T) := \mathbb{E}[ f(x_T, X_{\bar{T}}) | z]$  where $z$ is a hyperparameter

previous approaches considers
- unconditional: $z := \varnothing$
- conditional: $z := \{X_T = x_T \}$

we in addition consider
- do: $z := \{do(X_T = x_T)\}

## Assumptions
- no unobserved confounders

## Hyper parameters
- dropped_feature_imputation: unconditional, conditional, do 
- timelines: all, last, valid

## central questions

1. Does timeline matter

In [ ]:
class Node:
    def __init__(self, )
    
def dfs(node):
    if node.name == 'target':
        credit(node, node.val - node.last_val)
        return 
    
    for c in node.children:
        c.from = node
        
        dfs(c)